In [6]:
from merlion.evaluate.forecast import (
    ForecastEvaluator, ForecastEvaluatorConfig, ForecastMetric)
from merlion.models.factory import ModelFactory
from merlion.models.ensemble.combine import ModelSelector
from merlion.utils import TimeSeries as TS
from ts_datasets.forecast import SeattleTrail

In [7]:
ts, meta = SeattleTrail()[0]
train = TS.from_pd(ts[meta['trainval']])
test = TS.from_pd(ts[-meta['trainval']])

train.dim

5

In [8]:
target_seq_idx = 2
max_forecast_steps = 24
kwargs = {'target_seq_index': target_seq_idx,
          'max_forecast_steps': max_forecast_steps}

In [9]:
mod1 = ModelFactory.create('DefaultForecaster', **kwargs)
mod2 = ModelFactory.create('Arima', **kwargs)
mod3 = ModelFactory.create(
    'ForecasterEnsemble', 
    models=[mod1, mod2], 
    combiner=ModelSelector(metric=ForecastMetric.sMAPE))

In [10]:
for mod in [mod1, mod2, mod3]:
    print(f'Training {type(mod).__name__}...')
    preds, se = mod.train(train)

Training DefaultForecaster...


OSError: dlopen(/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [ ]:
target_uni = test.univariates9test.names[target_seq_idx]
target = target_uni[:max_forecast_steps].to_ts()

In [ ]:
for mod in [mod1, mod2, mod3]:
    forecast, se = mod.forecast(target.time_stamps)
    rmse = ForecastMetric.RMSE.value(groun_truth=target, predict=forecast)
    smape = ForecastMetric.sMAPE.value(groun_truth=target, 
                                       predict=forecast)
    print(f'{type(mod).__name__}\tRMSE: {rmse:.4f}\tsMAPE: {sMAPE:.4f})

In [ ]:
for mod in [mod1, mod2]:
    print(f'{type(mod).__name__} Sliding Winow Eval')
    evaluator = ForecastEvaluator(
        model=mod, 
        config=ForecastEvaluatorConfig(
            horizon='1d', cadence='1d', retrain_freq=None))
    train_res, test_pred = evaluator.get_predict(train_vals=train,
                                                 test_vals=test)
    rmse = evaluator.evaluate(
        ground_truth=test, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(
        ground_truth=test, predict=test_pred, metric=ForecastMetric.sMAPE)
    print(f'RMSE: {rmse:.4f}, sMAPE: {smape:.4f})